In [7]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd

In [3]:
df = pd.read_csv("E:/accident_severity_analytics/data/processed/Cleaned_US_Accidents.csv", parse_dates=["Start_Time", "End_Time", "Weather_Timestamp"])
df

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,...,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Duration
0,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,0.010,Accident,state route 32,...,False,False,False,True,False,Night,Night,Day,Day,30.000000
1,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,0.010,Accident,i-75,...,False,False,False,False,False,Night,Day,Day,Day,30.000000
2,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,0.010,Accident,miamisburg centerville road,...,False,False,False,True,False,Day,Day,Day,Day,30.000000
3,A-6,Source2,3,2016-02-08 07:44:26,2016-02-08 08:14:26,40.100590,-82.925194,0.010,Accident,westerville road,...,False,False,False,False,False,Day,Day,Day,Day,30.000000
4,A-7,Source2,2,2016-02-08 07:59:35,2016-02-08 08:29:35,39.758274,-84.230507,0.000,Accident,woodward avenue,...,False,False,False,False,False,Day,Day,Day,Day,30.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6335823,A-7777757,Source1,2,2019-08-23 18:03:25,2019-08-23 18:32:01,34.002480,-117.379360,0.543,Accident,pomona fwy,...,False,False,False,False,False,Day,Day,Day,Day,28.600000
6335824,A-7777758,Source1,2,2019-08-23 19:11:30,2019-08-23 19:38:23,32.766960,-117.148060,0.338,Accident,i-8,...,False,False,False,False,False,Day,Day,Day,Day,26.883333
6335825,A-7777759,Source1,2,2019-08-23 19:00:21,2019-08-23 19:28:49,33.775450,-117.847790,0.561,Accident,garden grove fwy,...,False,False,False,False,False,Day,Day,Day,Day,28.466667
6335826,A-7777760,Source1,2,2019-08-23 19:00:21,2019-08-23 19:29:42,33.992460,-118.403020,0.772,Accident,san diego fwy,...,False,False,False,False,False,Day,Day,Day,Day,29.350000


In [11]:
app = dash.Dash(__name__)

# تصميم الداش بورد
app.layout = html.Div([
    html.H1("Dashboard: تحليل الحوادث"),
    
    # فلتر الزمن (التاريخ)
    dcc.DatePickerRange(
        id='date-picker-range',
        start_date=df['Start_Time'].min().date(),  # استخدام Start_Time
        end_date=df['Start_Time'].max().date(),
        display_format='YYYY-MM-DD',
        style={'padding': '20px'}
    ),
    
    # الفلتر الخاص بالطقس
    dcc.Dropdown(
        id='weather-filter',
        options=[{'label': weather, 'value': weather} for weather in df['Weather_Condition'].unique()],
        multi=True,
        placeholder="اختر الطقس",
        style={'width': '50%'}
    ),
    
    # الفلتر الخاص بالموقع
    dcc.Dropdown(
        id='location-filter',
        options=[{'label': location, 'value': location} for location in df['City'].unique()],
        multi=True,
        placeholder="اختر الموقع",
        style={'width': '50%'}
    ),
    
    # رسم بياني للعدد الكلي للحوادث حسب التاريخ
    dcc.Graph(id='accident-time-chart'),

    # رسم بياني للعدد حسب الطقس
    dcc.Graph(id='accident-weather-chart'),

    # رسم بياني للموقع
    dcc.Graph(id='accident-location-chart'),
])

# دالة التحديث للرسوم البيانية بناءً على الفلاتر
@app.callback(
    [dash.dependencies.Output('accident-time-chart', 'figure'),
     dash.dependencies.Output('accident-weather-chart', 'figure'),
     dash.dependencies.Output('accident-location-chart', 'figure')],
    [dash.dependencies.Input('date-picker-range', 'start_date'),
     dash.dependencies.Input('date-picker-range', 'end_date'),
     dash.dependencies.Input('weather-filter', 'value'),
     dash.dependencies.Input('location-filter', 'value')]
)
def update_graphs(start_date, end_date, selected_weather, selected_location):
    # تصفية البيانات بناءً على الفلاتر
    filtered_df = df[(df['Start_Time'].dt.date >= pd.to_datetime(start_date).date()) & 
                     (df['Start_Time'].dt.date <= pd.to_datetime(end_date).date())]
    
    if selected_weather:
        filtered_df = filtered_df[filtered_df['Weather_Condition'].isin(selected_weather)]
    if selected_location:
        filtered_df = filtered_df[filtered_df['City'].isin(selected_location)]
    
    # رسم بياني للعدد الكلي للحوادث حسب التاريخ
    accident_time_chart = px.line(filtered_df, x='Start_Time', title="الحوادث عبر الزمن",
                                  labels={'Start_Time': 'التاريخ', 'count': 'عدد الحوادث'},
                                  category_orders={'Start_Time': sorted(filtered_df['Start_Time'].dt.date.unique())})
    
    # رسم بياني للعدد حسب الطقس
    accident_weather_chart = px.bar(filtered_df, x='Weather_Condition', title="الحوادث حسب الطقس",
                                    labels={'Weather_Condition': 'الطقس', 'count': 'عدد الحوادث'},
                                    category_orders={'Weather_Condition': sorted(filtered_df['Weather_Condition'].unique())})
    
    # رسم بياني للموقع
    accident_location_chart = px.bar(filtered_df, x='City', title="الحوادث حسب الموقع",
                                     labels={'City': 'الموقع', 'count': 'عدد الحوادث'},
                                     category_orders={'City': sorted(filtered_df['City'].unique())})
    
    return accident_time_chart, accident_weather_chart, accident_location_chart


app.run(jupyter_mode='external')

Dash app running on http://127.0.0.1:8050/


In [ ]:
app.run(jupyter_mode='external')


In [ ]:


# Initialize the Dash app

app = dash.Dash(**name**)

# Layout

app.layout = html.Div(\[
html.H1('Accident Analysis Dashboard', style={'textAlign': 'center', 'margin': '20px'}),

```
# Refresh button
html.Button('Refresh Data', id='refresh-button', style={'margin': '10px'}),

# Sidebar for filters
html.Div([
    html.H3('Filters', style={'textAlign': 'left'}),
    dcc.Dropdown(
        id='vehicle-type',
        options=[{'label': v, 'value': v} for v in df['vehicle type code 1'].unique()],
        placeholder='Select Vehicle Type'
    ),
    dcc.DatePickerRange(
        id='date-range',
        start_date=df['crash date'].min(),
        end_date=df['crash date'].max(),
        display_format='YYYY-MM-DD'
    ),
], style={'width': '20%', 'float': 'left', 'padding': '20px', 'backgroundColor': '#f0f0f0'}),

# Main content
html.Div([
    html.H2('Overview'),
    html.P(f'Total Accidents: {len(df)}'),
    html.P(f'Total Injuries: {df["number of persons injured"].sum()}'),
    html.P(f'Total Fatalities: {df["number of persons killed"].sum()}'),

    # Time Distribution Analysis
    html.H2('Time Distribution of Accidents'),
    dcc.Graph(id='time-distribution'),

    # High-Risk Locations Analysis
    html.H2('High-Risk Area Clustering'),
    dcc.Graph(id='cluster-map'),

    # Injuries and Fatalities Correlation
    html.H2('Injuries vs Fatalities'),
    dcc.Graph(id='injuries-vs-fatalities'),

    # Most Common Causes of Accidents
    html.H2('Most Common Accident Causes'),
    dcc.Graph(id='common-causes'),
], style={'width': '75%', 'float': 'right', 'padding': '20px'})
```

])

# Callbacks for data refresh

@app.callback(
dash.dependencies.Output('total-accidents', 'children'),
dash.dependencies.Output('total-injuries', 'children'),
dash.dependencies.Output('total-fatalities', 'children'),
dash.dependencies.Input('refresh-button', 'n\_clicks')
)
def update\_overview(n\_clicks):
refresh\_data()
return (
f'Total Accidents: {len(df)}',
f'Total Injuries: {df\["number of persons injured"].sum()}',
f'Total Fatalities: {df\["number of persons killed"].sum()}'
)

# Callbacks for interactivity

@app.callback(
dash.dependencies.Output('time-distribution', 'figure'),
\[dash.dependencies.Input('vehicle-type', 'value'),
dash.dependencies.Input('date-range', 'start\_date'),
dash.dependencies.Input('date-range', 'end\_date')]
)
def update\_time\_distribution(vehicle\_type, start\_date, end\_date):
filtered\_df = df\[(df\['crash date'] >= start\_date) & (df\['crash date'] <= end\_date)]
if vehicle\_type:
filtered\_df = filtered\_df\[filtered\_df\['vehicle type code 1'] == vehicle\_type]
fig = px.histogram(filtered\_df, x='crash date', title='Accident Count Over Time')
return fig

@app.callback(
dash.dependencies.Output('cluster-map', 'figure'),
\[dash.dependencies.Input('vehicle-type', 'value')]
)
def update\_cluster\_map(vehicle\_type):
filtered\_df = df
if vehicle\_type:
filtered\_df = df\[df\['vehicle type code 1'] == vehicle\_type]
fig = px.scatter\_map(filtered\_df, lat='latitude',
lon='longitude', color='number of persons injured',
size='number of persons injured',
title='High-Risk Area Clustering', zoom=10)

```
return fig
```

@app.callback(
dash.dependencies.Output('injuries-vs-fatalities', 'figure'),
\[dash.dependencies.Input('vehicle-type', 'value')]
)
def update\_injuries\_vs\_fatalities(vehicle\_type):
filtered\_df = df
if vehicle\_type:
filtered\_df = df\[df\['vehicle type code 1'] == vehicle\_type]
fig = px.scatter(filtered\_df, x='number of persons injured', y='number of persons killed', color='borough', title='Injuries vs Fatalities')
return fig

@app.callback(
dash.dependencies.Output('common-causes', 'figure'),
\[dash.dependencies.Input('vehicle-type', 'value')]
)
def update\_common\_causes(vehicle\_type):
filtered\_df = df
if vehicle\_type:
filtered\_df = df\[df\['vehicle type code 1'] == vehicle\_type]
cause\_counts = filtered\_df\['contributing factor vehicle 1'].value\_counts().head(10)
fig = px.bar(x=cause\_counts.index, y=cause\_counts.values, labels={'x': 'Cause', 'y': 'Count'}, title='Most Common Accident Causes')
return fig

app.run(jupyter\_mode='external')
